# Pathway pTA1_FASII

The pTA1_FASII is identical to the pYPK0_FASII but for the host vector pTA1 (LEU2) instead of pYPKpw (URA3).

In [1]:
from pydna.parsers import parse_primers
from pydna.readers import read
from pydna.amplify import pcr
from pydna.assembly import Assembly
from IPython.display import display

Initiate the standard primers needed to amplify each cassette.
The first cassette in the pathway is amplified with standard
primers 577 and 778, the last with
775 and 578 and all others with 775 and 778.
Standard primers are listed [here](standard_primers.txt).

In [2]:
p = { x.id: x for x in parse_primers("standard_primers.txt") }

Restriction enzymes are imported from the Biopython package.

In [3]:
from Bio.Restriction import EcoRV, NotI, PacI, FspAI

The backbone vector is linearized with [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html).

In [4]:
pTA1 = read("pTA1.gb")

The cassette_products variable holds the list of expression cassette PCR products fragments to
be assembled.

In [5]:
cassette_products = []

The expression cassettes comes from a series of single gene expression vectors
held in the template_vectors list.

In [6]:
cassette_vectors = ("pYPK0_PDC1_EcfabH_TEF1.gb",
                    "pYPK0_TEF1_EcfabD_FBA1.gb",
                    "pYPK0_FBA1_EcfabG_RPL22A.gb",
                    "pYPK0_RPL22A_EcacpP_TDH3.gb",
                    "pYPK0_TDH3_EcfabF_UTR2.gb",
                    "pYPK0_UTR2_EcfabB_TPI1.gb",
                    "pYPK0_TPI1_EcfabA_PMP3.gb",
                    "pYPK0_PMP3_EcfabZ_ENO2.gb",
                    "pYPK0_ENO2_Athmod1_RPL5.gb",
                    "pYPK0_RPL5_AthfatA1_RPL16A.gb",
                    "pYPK0_RPL16A_AthfatB_RPL17A.gb",
                    "pYPK0_RPL17A_EcacpH_RPL16B.gb",
                    "pYPK0_RPL16B_EcacpS_TMA19.gb")
template_vectors = [read(v.strip()) for v in cassette_vectors if v.strip()]
for tv in  template_vectors:
    display(tv)

File(id)(o8280)

File(id)(o7931)

File(id)(o7583)

File(id)(o7153)

File(id)(o8352)

...

File(id)(o7763)

File(id)(o7709)

File(id)(o7993)

File(id)(o7740)

File(id)(o7275)

The first cassette in the pathway is amplified with standard primers 577 and 778.

In [7]:
cassette_products.append( pcr( p['577'], p['778'],  template_vectors[0] ) )

Cassettes in the middle cassettes are amplified with standard primers 1123 and 778.

In [8]:
cassette_products.extend( pcr( p['1123'], p['778'], v) for v in template_vectors[1:-1] )

In [9]:
# pcr( p['1123'], p['512'],  template_vectors[4])

In [10]:
cassette_products[4] = pcr( p['508'], p['512'],  template_vectors[4])

In [11]:
cassette_products[5] = pcr( p['1026'], p['621'],  template_vectors[5])

The last cassette in the pathway is amplified with standard primers 1123 and 578.

In [12]:
cassette_products.append( pcr( p['1123'], p['578'], template_vectors[-1] ) )

The cassettes are given names based on their order in the final construct in the code cell below.

In [13]:
for i, cp in enumerate(cassette_products):
	cp.name = "Cassette {}".format(i+1)
	print(cp.name)

Cassette 1
Cassette 2
Cassette 3
Cassette 4
Cassette 5
Cassette 6
Cassette 7
Cassette 8
Cassette 9
Cassette 10
Cassette 11
Cassette 12
Cassette 13


Cassettes and plasmid backbone are joined by homologous recombination in a Saccharomyces cerevisiae ura3 host
which selects for the URA3 gene in pYPKpw.

In [14]:
asm = Assembly( [pTA1.linearize(FspAI)] + cassette_products, limit=167-47-10)
asm

Assembly
fragments..: 6175bp 2780bp 2282bp 1934bp 1504bp 2275bp 2534bp 2000bp 1875bp 2114bp 2060bp 2344bp 2091bp 1908bp
limit(bp)..: 110
G.nodes....: 28
algorithm..: common_sub_strings

Normally, only one circular product should be formed since the
homology limit is quite large (see cell above). More than one
circular products might indicate an incorrect strategy.
The largest recombination product is chosen as candidate.

In [15]:
candidate, *rest = asm.assemble_circular()

In [16]:
rest

[Contig(o26279)]

This assembly figure shows how the fragments came together.

In [17]:
candidate.figure()

 -|pTA1_lin|124
|           \/
|           /\
|           124|Cassette 1|593
|                          \/
|                          /\
|                          593|Cassette 2|644
|                                         \/
|                                         /\
|                                         644|Cassette 3|440
|                                                        \/
|                                                        /\
|                                                        440|Cassette 4|287
|                                                                       \/
|                                                                       /\
|                                                                       287|Cassette 5|634
|                                                                                      \/
|                                                                                      /\
|                                                

The plasmid origin is shifted so that it matches the backbone vector, starting with the 577 primer.

In [18]:
pw = candidate.synced("gttctgatcctcgagcatcttaagaattc")

In [23]:
assert len(pw) == 26279

The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](https://pydna.readthedocs.io/index.html?highlight=cseguid#pydna.utils.cseguid)
uniquely identifies a circular double stranded sequence.

In [24]:
assert pw.cseguid() == "GsaGCT5nmHM4ixNQocrG-EvFVAY"

The file is given a name based on the sequence of expressed genes.

In [25]:
pw.locus = "pTA1_FASII"
pw.definition = "pTA1 PDC1 EcfabH TEF1 EcfabD FBA1 EcfabG RPL22A EcacpP TDH3 EcfabF UTR2 EcfabB TPI1 EcfabA PMP3 EcfabZ ENO2 Athmod1 RPL5 AthfatA1 RPL16A AthfatB RPL17A EcacpH RPL16B EcacpS TMA19"

Stamp sequence with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [26]:
pw.stamp()

cSEGUID GsaGCT5nmHM4ixNQocrG-EvFVAY

Write sequence to a local file.

In [27]:
pw.write("pTA1_FASII.gb")

pTA1_FASII.gb

The pathway can be extended by digestion with either NotI or PacI or both provided that the enzymes cut once in the final pathway sequence.